In [1]:
import os
import csv
import string
import random
import numpy as np
import pandas as pd
from scipy import optimize as optim
from scipy.stats import norm
from scipy.stats import mode
from scipy.stats import bartlett
from scipy.stats import sem
from scipy.stats import ttest_ind
from statistics import NormalDist
import matplotlib.pyplot as plt
import matplotlib as mpl
from openpyxl import load_workbook

#function to open data from excel
def opendata(openfilename, ticker, packagesize):
    #opening data from excel
    df = pd.read_excel(openfilename, sheet_name = ticker) 

    #converting data to numpy arrays
    df = df.to_numpy()
    
    #ms data too big to plot, have to cut the last 50000 data out
    if ticker == 'ms':
        cut = 50000
        crop = df[:-cut,0].size%packagesize
        df_size = df[0:df[:,1].size-crop-cut,1]
        df_time = df[0:df[:,2].size-crop-cut,2]
        df_type = df[0:df[:,3].size-crop-cut,3]
        df_values = df[0:df[:,4].size-crop-cut,4]
        #df_size = np.delete(df[: , 1], range(df[:,1].size-crop-cut, df[:,1].size-cut), None)
        #df_time = np.delete(df[: , 2], range(df[:,2].size-crop-cut, df[:,2].size-cut), None)
        #df_type = np.delete(df[: , 3], range(df[:,3].size-crop-cut, df[:,3].size-cut), None)
        #df_values = np.delete(df[: , 4], range(df[:,4].size-crop-cut, df[:,4].size-cut), None)
        
        
    else:
        #cropping our data, so it can be divided into five-set packages and dividing data to multiple arrays
        crop = df[:,0].size%packagesize

        df_size = np.delete(df[: , 1], range(df[:,1].size-crop, df[:,1].size), None)
        df_time = np.delete(df[: , 2], range(df[:,2].size-crop, df[:,2].size), None)
        df_type = np.delete(df[: , 3], range(df[:,3].size-crop, df[:,3].size), None)
        df_values = np.delete(df[: , 4], range(df[:,4].size-crop, df[:,4].size), None)
    
    
    return df_size, df_time, df_type, df_values;

def predict_V_and_P(openfilename, ticker, packagesize, SIG_u, sample):
    
    df_size, df_time, df_type, df_values = opendata(openfilename = openfilename, ticker= ticker, packagesize = packagesize)

    #SIG_u is the sigma value of quantity traded by noise traders
    #creating the original bid-ask spread
    #dividing the original size value into quantities traded by insiders, and quantities traded by noise traders
    df_spread = np.zeros((df_values.size, 3))
    df_usize = np.zeros(df_values.size)
    df_isize = np.zeros(df_values.size)
    for x in range(0, df_values.size):
        if x == 0 and ticker == 'ms':
            df_spread[0, 1] = df_values[0]
            df_spread[0, 0] = df_values[1]
            df_spread[0, 2] = (df_spread[0, 0]+df_spread[0, 1])/2
            df_usize[x] = np.round_(np.random.normal(0, SIG_u))
            df_isize[x] = df_size[x]-df_usize[x]
        elif x== 0 and ticker == 'otp':
            df_spread[0, 0] = df_values[0]
            df_spread[0, 1] = df_values[1]
            df_spread[0, 2] = (df_spread[0, 0] + df_spread[0, 1])/2
            df_usize[x] = np.round(np.random.normal(0, SIG_u))
            df_isize[x] = df_size[x]-df_usize[x]
        else:
            if df_type[x] == "BEST_ASK":
                df_spread[x, 1]= df_values[x]
                df_spread[x, 0]= df_spread[x-1, 0]
                df_spread[x, 2]= (df_spread[x, 0]+df_spread[x, 1])/2
                df_usize[x] = np.round_(np.random.normal(0, SIG_u))
                df_isize[x] = df_size[x]-df_usize[x]
            elif df_type[x] == "BEST_BID":
                df_spread[x, 1]= df_spread[x-1, 1]
                df_spread[x, 0]= df_values[x]
                df_spread[x, 2]= (df_spread[x, 0]+df_spread[x, 1])/2
                df_usize[x] = np.round(np.random.normal(0, SIG_u))
                df_isize[x] = df_size[x]-df_usize[x]
    
    if packagesize == 1 or packagesize == 0:
        
        df_price_predict = np.zeros(df_spread[:, 2].size-sample)
        df_size_predict = np.zeros(df_size.size-sample)
    
        for x in range(0, df_price_predict.size):
        
            #calculating standard deviation of v on the given sample
        
            vmu = np.mean(df_spread[x:x+sample-1, 2]) 
            vstd = np.std(df_spread[x:x+sample-1, 2])
        
            #calculating beta and lambda values
            beta = SIG_u**2 / vstd**2
            Lambda = 2*np.sqrt(SIG_u**2 / vstd**2)
        
            #predicting all traded volume by predicting insider traded volume and giving to it a randomly generated noise trader volume
            df_size_predict[x] = np.round_((beta*(df_spread[x+sample-1,2]-vmu)) + np.round(np.random.normal(0, SIG_u)))
            df_price_predict[x] = vmu + Lambda*df_size_predict[x]
    
        for x in range(0, df_price_predict.size):
            if (np.isnan(df_size_predict[x]) == True or np.isfinite(df_size_predict[x]) == False):
                df_size_predict[x] = 1
            
            if (np.isnan(df_price_predict[x]) == True or np.isfinite(df_price_predict[x]) == False):
                df_price_predict[x] = 1
    
        return df_size_predict, df_price_predict, df_size, df_spread[:,2];
    
    else:
        
        #creating the packagesize sized packages, summing the total amount of trading shares, and averaging the bid-ask spreads in those packages
        df5_spread = np.zeros((df_spread[:,0].size//packagesize, 3))
        for j in range(0, df_spread[0,:].size):
            for x in range(0, df5_spread[:,j].size):
                df5_spread[x,j] = np.mean(df_spread[x*packagesize:(((x+1)*packagesize)-1), j])

        df5_size = np.zeros(df_size.size//packagesize)
        for x in range(0, df5_size.size):
            df5_size[x] = np.round_(np.mean(df_size[x*packagesize:(((x+1)*packagesize)-1)]))
    
        df5_usize = np.zeros(df_size.size//packagesize)
        df5_isize = np.zeros(df_size.size//packagesize)
        for x in range(0, df5_usize.size):
            df5_usize[x] = np.round_(np.mean(df_usize[x*packagesize:(((x+1)*packagesize)-1)]))
            df5_isize[x] = np.round_(np.mean(df_isize[x*packagesize:(((x+1)*packagesize)-1)]))

        #creating arrays into which the predicted values will go
        df5_price_predict = np.zeros(df5_spread[:, 2].size-sample)
        df5_size_predict = np.zeros(df5_size.size-sample)
    
        for x in range(0, df5_price_predict.size):
        
            #calculating standard deviation of v on the given sample
            
            vmu = np.mean(df5_spread[x:x+sample-1, 2]) 
            vstd = np.std(df5_spread[x:x+sample-1, 2])
        
            #calculating beta and lambda values
            beta = SIG_u**2 / vstd**2
            Lambda = 2*np.sqrt(SIG_u**2 / vstd**2)
        
            #predicting all traded volume by predicting insider traded volume and giving to it a randomly generated noise trader volume
            df5_size_predict[x] = np.round_((beta*(df5_spread[x+sample-1,2]-vmu)) + np.round(np.random.normal(0, SIG_u)))
            df5_price_predict[x] = vmu + Lambda*df5_size_predict[x]
    
        for x in range(0, df5_price_predict.size):
            if (np.isnan(df5_size_predict[x]) == True or np.isfinite(df5_size_predict[x]) == False):
                df5_size_predict[x] = 1
            
            if (np.isnan(df5_price_predict[x]) == True or np.isfinite(df5_price_predict[x]) == False):
                df5_price_predict[x] = 1
        
        return df5_size_predict, df5_price_predict, df5_size, df5_spread[:,2];

    

def results(pred_size, pred_price, org_size, org_value, outfilename, testname):

    res = np.zeros(12)
    diff = org_size.size - pred_size.size
    
    
    #calculating indicators for predicted and original size 
    #original average size value
    res[1] = np.mean(org_size[diff:])
    #predicted average size value
    res[0] = np.mean(pred_size)
    
    res[3], trash = mode(org_size[diff:])
    res[2], trash = mode(pred_size)
    
    maxdiff = 0
    differences = np.zeros(org_size[diff:].size)
    for x in range(0, org_size[diff:].size):
        differences[x] = np.absolute(pred_size[x] - org_size[x+diff])
        if differences[x] > maxdiff:
            maxdiff = differences[x]
    
    #average difference of sizes
    res[4] = np.mean(differences)
    #max difference
    res[5] = maxdiff
    
    #calculating indicators for predicted and original price values
    #calculating expected value and  standard deviation from original spread,
    res[7] = np.mean(org_value[diff:])
    res[9] = np.std(org_value[diff:])
    
    #calculating expected value and  standard deviation from predicted prices
    res[6] = np.mean(pred_price)
    res[8] = np.std(pred_price)
    
    #calculating average and max differences between predicted and original price values
    
    maxdiff = 0
    differences = np.zeros(org_value[diff:].size)
    for x in range(0, org_value[diff:].size):
        differences[x] = np.absolute(pred_price[x] - org_value[x+diff])
        if differences[x] > maxdiff:
            maxdiff = differences[x]
    
    res[10] = np.mean(differences)
    res[11] = maxdiff
    
    mpl.rcParams['agg.path.chunksize'] = 10000

    #ploting the original value and the predicted price, then saving them
    plt.subplot(2, 1, 1)
    plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=1.2)
    plt.plot(pred_price, 'b.')
    plt.xlabel('Number of trades')
    plt.ylabel('Predicted price')
    plt.yscale('linear')
    #plt.savefig(testname+'_p_pred', dpi=1000)
    #matplotlib.pylab.close(fig)
    #plt.clf()
    
    plt.subplot(2, 1, 2)
    fig =plt.plot(org_value[diff:], 'r.')
    plt.xlabel('Number of trades')
    plt.ylabel('Original average spread')
    plt.yscale('linear')
    plt.savefig(testname+'_p', dpi=1000)
    #matplotlib.pylab.close(fig)
    plt.clf()
    
    #ploting the original and predicted traded volumes, then saving them
    plt.subplot(2, 1, 1)
    plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=1.2)
    #plt.title('Results of test: ' + testname)
    plt.plot(pred_size, 'b.')
    plt.xlabel('Number of trades')
    plt.ylabel('Predicted amount traded')
    plt.yscale('linear')
    #plt.savefig(testname + '_v_pred', dpi=1000)
    #matplotlib.pylab.close(fig)
    #plt.clf()
    
    plt.subplot(2, 1, 2)
    plt.plot(org_size[diff:], 'r.')
    plt.xlabel('Number of trades')
    plt.ylabel('Original amount traded')
    plt.yscale('linear')
    plt.savefig(testname + '_v', dpi=1000)
    #matplotlib.pylab.close(fig)
    plt.clf()
    
    #saving predicted values into xlsx
    book = load_workbook(outfilename)
    writer = pd.ExcelWriter(outfilename, engine = 'openpyxl')
    writer.book = book

    toexcel = pd.DataFrame(data = {'PRED_size': pred_size, 'PRED_PRICE' : pred_price} )
    toexcel.to_excel(writer, sheet_name = testname)
    
    writer.save()
    writer.close()
    
    return res


ind = np.zeros((12, 14))
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 4, SIG_u = 6, sample = 100)
ind[:,0] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_1')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 8, SIG_u = 6, sample = 100)
ind[:,1] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_2')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 2, SIG_u = 6, sample = 100)
ind[:,2] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_3')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 4, SIG_u = 6, sample = 200)
ind[:,3] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_4')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 4, SIG_u = 6, sample = 50)
ind[:,4] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_5')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 4, SIG_u = 12, sample = 100)
ind[:,5] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_6')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'ms', packagesize = 4, SIG_u = 3, sample = 100)
ind[:,6] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_1_7')

pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 4, SIG_u = 6, sample = 100)
ind[:,7] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_1')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 8, SIG_u = 6, sample = 100)
ind[:,8] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_2')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 2, SIG_u = 6, sample = 100)
ind[:,9] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_3')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 4, SIG_u = 6, sample = 200)
ind[:,10] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_4')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 4, SIG_u = 6, sample = 50)
ind[:,11] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_5')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 4, SIG_u = 12, sample = 100)
ind[:,12] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_6')
pred_size, pred_price, org_size, org_value = predict_V_and_P(openfilename = 'data.xlsx', ticker = 'otp', packagesize = 4, SIG_u = 3, sample = 100)
ind[:,13] = results(pred_size, pred_price, org_size, org_value, outfilename = 'Single_Step_Kyle_Model_Test_Results.xlsx', testname = 'SK_2_7')

toout = pd.DataFrame(data = {'Predicted mean value': ind[0,:], 'Original mean volume' : ind[1,:], 'Predicted mode volume' : ind[2,:], 'Original mode volume' : ind[3,:], 'Mean difference' : ind[4,:], 'Max difference' : ind[5,:], 'Predicted prices mean' : ind[6,:], 'Original value mean' : ind[7,:], 'Standard deviation of predicted prices' : ind[8,:], 'Standard deviation of original value' : ind[9,:], 'Mean of differences' : ind[10,:], 'Max difference' : ind[11,:]} )
toout.to_excel('SK_indicators.xlsx')


<ipython-input-1-522c5dc1aa66>:148: RuntimeWarning: divide by zero encountered in double_scalars
  beta = SIG_u**2 / vstd**2
<ipython-input-1-522c5dc1aa66>:149: RuntimeWarning: divide by zero encountered in double_scalars
  Lambda = 2*np.sqrt(SIG_u**2 / vstd**2)
<ipython-input-1-522c5dc1aa66>:152: RuntimeWarning: invalid value encountered in double_scalars
  df5_size_predict[x] = np.round_((beta*(df5_spread[x+sample-1,2]-vmu)) + np.round(np.random.normal(0, SIG_u)))
<ipython-input-1-522c5dc1aa66>:148: RuntimeWarning: divide by zero encountered in double_scalars
  beta = SIG_u**2 / vstd**2
<ipython-input-1-522c5dc1aa66>:149: RuntimeWarning: divide by zero encountered in double_scalars
  Lambda = 2*np.sqrt(SIG_u**2 / vstd**2)
<ipython-input-1-522c5dc1aa66>:152: RuntimeWarning: invalid value encountered in double_scalars
  df5_size_predict[x] = np.round_((beta*(df5_spread[x+sample-1,2]-vmu)) + np.round(np.random.normal(0, SIG_u)))
<ipython-input-1-522c5dc1aa66>:148: RuntimeWarning: divide

<Figure size 432x288 with 0 Axes>